In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
!cp /kaggle/input/* /kaggle/working -r

In [3]:
cats_dir = "/kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Cat"
dogs_dir = "/kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Dog"

In [4]:
img_size = 256
apply_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

In [5]:
def removeCorruptImages(paths):
    for path in paths:
        for filename in os.listdir(path):  
            try:
                Image.open(os.path.join(path, filename))  
            except (IOError, OSError):  
                print(f"Removed: {os.path.join(path, filename)}")
                os.remove(os.path.join(path, filename))  


In [6]:
removeCorruptImages([cats_dir, dogs_dir])

Removed: /kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg
Removed: /kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Cat/Thumbs.db
Removed: /kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Dog/Thumbs.db


/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed: /kaggle/working/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg


In [7]:
class LazyImgLoader(Dataset):
    def __init__(self, path, label, transforms=None):
        self.path = path
        self.label = label
        self.transforms = transforms
        self.total_len = len(os.listdir(path))

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.path, os.listdir(self.path)[idx]))
        img = img.convert("RGB")
        if self.transforms:
            img = self.transforms(img)
        return img, self.label

In [8]:
cat_dataset = LazyImgLoader(cats_dir, 0, transforms = apply_transform)
dog_dataset = LazyImgLoader(dogs_dir, 1, transforms = apply_transform)

In [9]:
combined_dataset = ConcatDataset([cat_dataset, dog_dataset])
del cat_dataset
del dog_dataset

In [10]:
train_set, test_set = train_test_split(combined_dataset, shuffle = True, test_size = 0.2)
del combined_dataset

In [11]:
train_set, val_set = train_test_split(train_set, shuffle = True, test_size = 0.12)

In [12]:
batch_size = 64
in_channels = 3
learning_rate = 3e-4
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [13]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_set, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [14]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=2):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels = in_channels, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1) ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2)),
            
            nn.Conv2d(in_channels = 8, out_channels = 32, kernel_size = (3,3), stride = (1, 1), padding = (1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2)),
                        
            nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = (3,3), stride = (1, 1), padding = (1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2)),
        )
    
        self.lin_layers = nn.Sequential(
            nn.Linear(16 * 32 * 32, 4096),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(4096,1024),
            nn.ReLU(),
            nn.Linear(1024, 128),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.Dropout(0.4),
            nn.Linear(32, 2),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.lin_layers(x)
        return x


In [15]:
model = CNN().to(device)
model = torch.nn.DataParallel(model)


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [17]:
for epoch in range(num_epochs):
    epoch_loss = 0;
    print(epoch if epoch % 5 == 0 else print(""))
    model.train()
    for (data, labels) in train_loader:
        data = data.to(device)
        labels = labels.to(device)

        scores = model(data)
        loss = criterion(scores, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
            
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for (data, labels) in val_loader:
            data = data.to(device)
            labels = labels.to(device)
            
            scores = model(data)
            loss = criterion(scores, labels)
            
            val_loss += loss.item()
            _, pred = scores.max(1)
            total += labels.size(0)
            correct += (labels == pred).sum()
            
        val_loss /= len(val_loader)
        print(f"Validation loss: ",{val_loss})
#     scheduler.step(val_loss)
    

0
Validation loss:  {0.6522720424752486}

None
Validation loss:  {0.5928411201426858}

None
Validation loss:  {0.5467241265271839}

None
Validation loss:  {0.5118651052838877}

None
Validation loss:  {0.49458622226589605}


In [18]:
def check_accuracy(model, loader):
    num_samples = 0
    num_correct = 0
    
    model.eval()
    
    with torch.no_grad():
        for data, labels in loader:
            data = data.to(device)
            labels = labels.to(device)
            
            scores = model(data)
            
            _, pred = scores.max(1)
            num_correct += (pred == labels).sum().item()
            num_samples += pred.size(0)
    print(f"Accuracy : {(num_correct / num_samples)*100:.2f}%")

In [19]:
check_accuracy(model, train_loader)

Accuracy : 81.44%


In [20]:
check_accuracy(model, val_loader)

Accuracy : 75.42%


In [21]:
check_accuracy(model, test_loader)

Accuracy : 75.82%
